<a href="https://colab.research.google.com/github/aayushkubb/nlp/blob/main/QUORA_question_is_sincere_or_not.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?id=1qyIqgII4yWXEkTC03x_sNbPeZa6MRWRJ"/>

Problem Definition
---
I think one of the important things when you start a new machine learning project is Defining your problem. that means you should understand business problem.( Problem Formalization)

> We will be predicting whether a question asked on Quora is sincere or not

Data Source : https://www.kaggle.com/c/quora-insincere-questions-classification/data

**About Quora**

Quora is a platform that empowers people to learn from each other. On Quora, people can ask questions and connect with others who contribute unique insights and quality answers. A key challenge is to weed out insincere questions -- those founded upon false premises, or that intend to make a statement rather than look for helpful answers.

**Business View**

An existential problem for any major website today is how to handle toxic and divisive content. Quora wants to tackle this problem head-on to keep their platform a place where users can feel safe sharing their knowledge with the world.

**What is a insincere question?**

Is defined as a question intended to make a statement rather than look for helpful answers.

<img src="https://drive.google.com/uc?id=1XOvVoEDjNF_p2tHV1H9XzoZHxfzBBA5t"/>

**Feature Set**

We use train.csv and test.csv as Input and we should upload a submission.csv as Output.

The training set contains the following 3 features (for Supervised Learning)
1. qid - unique question identifier
2. question_text - Quora question text
3. target - a question labeled "insincere" has a value of 1, otherwise 0

In [ ]:
pip install nltk

**Coding a solutiuon for solving the above problem**

In [ ]:
# all import statements
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
#from wordcloud import WordCloud as wc   # not needed
from nltk.corpus import stopwords
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
from pandas import get_dummies
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib
import warnings
import sklearn
import string
import scipy
import numpy
import nltk
import json
import sys
import csv
import os

In [ ]:
# printing versions of the important packages
print('matplotlib: {}'.format(matplotlib.__version__))
print('sklearn: {}'.format(sklearn.__version__))
print('scipy: {}'.format(scipy.__version__))
print('seaborn: {}'.format(sns.__version__))
print('pandas: {}'.format(pd.__version__))
print('numpy: {}'.format(np.__version__))
print('Python: {}'.format(sys.version))

matplotlib: 3.2.2
sklearn: 0.22.2.post1
scipy: 1.4.1
seaborn: 0.11.1
pandas: 1.1.5
numpy: 1.19.5
Python: 3.7.10 (default, Feb 20 2021, 21:17:23) 
[GCC 7.5.0]


We would first do EDA ( Exploratory Data Analysis ) over Quora Data set :
--

<img src="https://drive.google.com/uc?id=1uvFGaW5OUQGbNphJqBIlaIg7Z2xIXrfX"/>

In [ ]:
# Download datasets from the below given links

# QuoratrainSet.csv
#  https://drive.google.com/open?id=1EBNVP7zikYFfCz0sW1hTT516Gqb8TnKG

# Quoratestdata.csv
#  https://drive.google.com/open?id=1Ep-bFgiHUBdT8gOpVDuYc7lsiAS6u7gE


In [ ]:
from google.colab import drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
train=pd.read_csv('/content/drive/MyDrive/DSTRAIN/NLP/QuoratrainSet.csv')
test=pd.read_csv('/content/drive/MyDrive/DSTRAIN/NLP/Quoratestdata.csv')

In [ ]:
# check top 5 records of training dataset

train.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [ ]:
# check top 5 records of testing dataset
test.head()

,qid,question_text
0,0000163e3ea7c7a74cd7,Why do so many women become so rude and arroga...
1,00002bd4fb5d505b9161,When should I apply for RV college of engineer...
2,00007756b4a147d2b0b3,What is it really like to be a nurse practitio...
3,000086e4b7e1c7146103,Who are entrepreneurs?
4,0000c4c3fbe8785a3090,Is education really making good people nowadays?


In [ ]:
# Find the type of features in Quora dataset
# i.e get a quick statistics
train.dtypes


qid              object
question_text    object
target            int64
dtype: object

In [ ]:
print(test.info())  # see carefully the last value is -> None. 
                    # indicating that there are no "Null" values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375806 entries, 0 to 375805
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   qid            375806 non-null  object
 1   question_text  375806 non-null  object
dtypes: object(2)
memory usage: 5.7+ MB
None


In [ ]:
# shape for train and test
print('Shape of train:',train.shape)
print('Shape of test:',test.shape)

Shape of train: (1048575, 3)
Shape of test: (375806, 2)


**Data Cleaning**

When dealing with real-world data, dirty data is the norm rather than the exception. 

The primary goal of data cleaning is to detect and remove errors and anomalies to increase the value of data in analytics and decision making. While it has been the focus of many researchers for several years, individual problems have been addressed separately. These include missing value imputation, outliers detection, transformations, integrity constraints violations detection and repair, consistent query answering, deduplication, and many other related problems such as profiling and constraints mining.

In [ ]:
# How many NA elements in every column!!
# Good news, it is Zero!
# To check out how many null info are on the dataset, we can use isnull().sum().
# recall from info() -> we found that it has zero Nulls. 

train.isnull().sum()

# data is infact clean and ready for use.

qid              0
question_text    0
target           0
dtype: int64

In [ ]:
# in case , their were NA or None values in any row then we would drop the row.

# remove rows that have NA's
print('Before Droping',train.shape)
train = train.dropna()
print('After Droping',train.shape)

Before Droping (1048575, 3)
After Droping (1048575, 3)


In [ ]:
# Number of words in the text

train["num_words"] = train["question_text"].apply(lambda x: len(str(x).split()))
test["num_words"] = test["question_text"].apply(lambda x: len(str(x).split()))
print('maximum of num_words in train',train["num_words"].max())
#print('min of num_words in train',train["num_words"].min())
print("maximum of  num_words in test",test["num_words"].max())
#print('min of num_words in train',test["num_words"].min())

maximum of num_words in train 134
maximum of  num_words in test 87


In [ ]:
# Number of unique words in the text
train["num_unique_words"] = train["question_text"].apply(lambda x: len(set(str(x).split())))
test["num_unique_words"] = test["question_text"].apply(lambda x: len(set(str(x).split())))

print('maximum of num_unique_words in train',train["num_unique_words"].max())

print("maximum of num_unique_words in test",test["num_unique_words"].max())


maximum of num_unique_words in train 96
maximum of num_unique_words in test 61


In [ ]:
# Number of stopwords in the text

#from nltk.corpus import stopwords
eng_stopwords = set(stopwords.words("english"))

train["num_stopwords"] = train["question_text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
test["num_stopwords"] = test["question_text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))

print('maximum of num_stopwords in train',train["num_stopwords"].max())
print("maximum of num_stopwords in test",test["num_stopwords"].max())


LookupError: ignored

In [ ]:
# Number of punctuations in the text

train["num_punctuations"] =train['question_text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )
test["num_punctuations"] =test['question_text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )
print('maximum of num_punctuations in train',train["num_punctuations"].max())
print("maximum of num_punctuations in test",test["num_punctuations"].max())

In [ ]:
# lets figure out how many unique target values exist.
# like we expect : 0 -> sincere qns and 1 -> in-sincere qns

# You see number of unique item for Target with command below:
train_target = train['target'].values

np.unique(train_target)

**YES, quora problem is a binary classification!**

**The data is absolutely clean. But is it balanced ?**

I mean do we have equal no. of sincere and un-sincere questions in the dataset.

lets find out :

In [ ]:
# check no. of insincere Qns 
train.where(train ['target']==1).count()

**This means out of ____ records __ are unsincere**


In [ ]:
print("% of un-sincere qns is ", 86/1500)

This means we have approx. 94% of the sincere Qns.

**Such a dataset is imbalanced.**

>Imbalanced dataset is relevant primarily in the context of supervised machine learning involving two or more classes.

> Imbalance means that the number of data points available for different classes is different: If there are two classes, then balanced data would mean 50% points for each of the class. For most machine learning techniques, little imbalance is not a problem. So, if there are 60% points for one class and 40% for the other class, it should not cause any significant performance degradation. Only when the class imbalance is high, e.g. 90% points for one class and 10% for the other, standard optimization criteria or performance measures may not be as effective and would need modification.

<img src="https://drive.google.com/uc?id=1_vLZmDjxkdhTVgGm7AQedcPD2D6XnQKJ" />

A typical example of imbalanced data is encountered in e-mail classification problem where emails are classified into ham or spam. The number of spam emails is usually lower than the number of relevant (ham) emails. So, using the original distribution of two classes leads to imbalanced dataset.

Using accuracy as a performace measure for highly imbalanced datasets is not a good idea. For example, if 90% points belong to the true class in a binary classification problem, a default prediction of true for all data points leads to a classifier which is 90% accurate, even though the classifier has not learnt anything about the classification problem at hand!

In [ ]:
# visualising the imbalance in data set

ax=sns.countplot(x='target',hue="target", data=train  ,linewidth=5,edgecolor=sns.color_palette("dark", 3))
plt.title('Is data set imbalance?');

Data Preprocessing
----

This basically involves transforming raw data into an understandable format for NLP models.

Remember, our feature "Question_Text" is Text or String Object and No ML algo say KNN or Bayes classification would accept Text. Hence Pre-Processing is mandatory in this case.  

Below, I have recalled the two most important techniques that are also performed besides other easy to understand steps in data pre-processing:

1. Tokenization: This is a process of breaking a stream of text up into words, phrases, symbols, or other meaningful elements called tokens. The list of tokens becomes input for further processing. NLTK Library has word_tokenize and sent_tokenize to easily break a stream of text into a list of words or sentences, respectively.


2. Word Stemming/Lemmatization: The aim of both processes is the same, reducing the inflectional forms of each word into a common base or root. Lemmatization is closely related to stemming. The difference is that a stemmer operates on a single word without knowledge of the context, and therefore cannot discriminate between words which have different meanings depending on part of speech. However, stemmers are typically easier to implement and run faster, and the reduced accuracy may not matter for some applications.

<img src="https://drive.google.com/uc?id=1t6UR1rmEeLaVuscqA6SZTFMb3nCzYleh" />

**We would atleast the following (Text) Pre-Processing steps**

1. Change all the text to lower case

2. Word Tokenization

3. Remove Stop words

4. Remove Non-alpha text

5. Word Lemmatization

6. Converting the text data into Numeric vectors( called Vectorization )

In [ ]:
# step 1: Change all the text to lower case. 

# This is required as python interprets 'quora' and 'QUORA' differently

train['question_text'] = [entry.lower() for entry in train['question_text']]

test['question_text'] = [entry.lower() for entry in test['question_text']]

#test.head()

In [ ]:
# more imports for NLP
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
# step 2 : Tokenization : In this each entry in the corpus will be broken 
#                         into set of words

train['question_text']= [word_tokenize(entry) for entry in train['question_text']]

test['question_text']= [word_tokenize(entry) for entry in test['question_text']]

train.head()

In [ ]:
# Set random seed
# This is used to reproduce the same result every time 
# if the script is kept consistent otherwise each run 
# will produce different results. The seed can be set to any number.
np.random.seed(500)

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

In [ ]:
import nltk
nltk.download('wordnet')

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
# step 3, 4 and 5
# Remove Stop words and Numeric data 
# and perfom Word Stemming/Lemmenting.

# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb
# or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
# the tag_map would map any tag to 'N' (Noun) except
# Adjective to J, Verb -> v, Adverb -> R
# that means if you get a Pronoun then it would still be mapped to Noun


for index,entry in enumerate(train['question_text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    
    # pos_tag function below will provide the 'tag' 
    # i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only 
        # alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
            
    # The final processed set of words for each iteration will be stored 
    # in 'question_text_final'
    train.loc[index,'question_text_final'] = str(Final_words)  
    
print(train.head())

In [ ]:
# step 3, 4 and 5
# Remove Stop words and Numeric data 
# and perfom Word Stemming/Lemmenting.

# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb
# or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
# the tag_map would map any tag to 'N' (Noun) except
# Adjective to J, Verb -> v, Adverb -> R
# that means if you get a Pronoun then it would still be mapped to Noun


for index,entry in enumerate(test['question_text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words_test = []
    
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    
    # pos_tag function below will provide the 'tag' 
    # i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only 
        # alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words_test.append(word_Final)
            
    # The final processed set of words for each iteration will be stored 
    # in 'question_text_final'
    test.loc[index,'question_text_final'] = str(Final_words_test)    

print(test.head())

In [ ]:
Tfidf_vect = TfidfVectorizer()
Tfidf_vect.fit(train['question_text_final'])

Train_X_Tfidf = Tfidf_vect.transform(train['question_text_final'])

Test_X_Tfidf = Tfidf_vect.transform(test['question_text_final'])

In [ ]:
# You can use the below syntax to see the vocabulary that 
# it has learned from the corpus
print(Tfidf_vect.vocabulary_)

In [ ]:
print(Train_X_Tfidf)

# Output: 
# 1: Row number of ‘Train_X_Tfidf’, 
# 2: Unique Integer number of each word, 
# 3: Score calculated by TF-IDF Vectorizer

In [ ]:
print(Test_X_Tfidf)

Data Pre-processing is over !!
---

Use ML Algorithms to Predict the outcome
---

In [ ]:
# fit the training dataset on the MultinomialNB classifier
Naive=naive_bayes.MultinomialNB()

train_Y=train['target']
Naive.fit(Train_X_Tfidf,train_Y)

#predict the labels on validation dataset
predictions_NB=Naive.predict(Test_X_Tfidf)

print(predictions_NB)

print(numpy.unique(predictions_NB,return_counts=True))

In [ ]:
# to find the insincrere Qns according to our MultinominalNB model
test['predictionNB']=predictions_NB

QnsNB=test.loc[test['predictionNB']==1]

print(QnsNB.head())

In [ ]:
# just checking the MultinomialNB accuracy - approx.

# predict the labels on first 5000 records of the train dataset
testing_NB=Naive.predict(Train_X_Tfidf[:5000])
labels_NB=train.target[:5000]

from sklearn.metrics import accuracy_score
accuracy_score(testing_NB,labels_NB)

In [ ]:
#check for other metrics as the data is imbalanced
from sklearn import metrics

print(metrics.classification_report(testing_NB, labels_NB))
print(metrics.confusion_matrix(testing_NB,labels_NB))
print(metrics.precision_score(testing_NB,labels_NB))
print(metrics.recall_score(testing_NB,labels_NB))

### SVM

In [ ]:
# Use another Classifier Algorithm say SVM
# fit the training dataset on the classifier

SVM=svm.SVC(C=1.0,kernel='linear',degree=3,gamma='auto')

SVM.fit(Train_X_Tfidf,train['target'])

#predict the labels on validation dataset
predictions_SVM=SVM.predict(Test_X_Tfidf)

print(predictions_SVM)

print(numpy.unique(predictions_SVM,return_counts=True))

In [ ]:
# to find the insincrere Qns according to our model
test['predictions']=predictions_SVM

QnsSVM=test.loc[test['predictions']==1]

print(QnsSVM.question_text.head())

In [ ]:
# just checking the SVM accuracy - approx.
# predict the labels on first 5000 records of the train dataset

testing_SVM=SVM.predict(Train_X_Tfidf[:5000])
labels=train.target[:5000]

from sklearn.metrics import accuracy_score
print(accuracy_score(testing_SVM,labels))

#check for other metrics as the data is imbalanced
from sklearn import metrics
print(metrics.classification_report(testing_SVM, labels))
print(metrics.confusion_matrix(testing_SVM,labels))
print(metrics.precision_score(testing_SVM,labels))
print(metrics.recall_score(testing_SVM,labels))

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()

rf.fit(Train_X_Tfidf,train['target'])

#predict the labels on validation dataset
predictions_rf=rf.predict(Test_X_Tfidf)

In [ ]:
# to find the insincrere Qns according to our model
test['predictions_rf']=predictions_rf

QnsRF=test.loc[test['predictions_rf']==1]

print(QnsRF.question_text.head())

In [ ]:
# just checking the random accuracy - approx.
# predict the labels on first 5000 records of the train dataset

testing_rf=rf.predict(Train_X_Tfidf[:5000])
labels_rf=train.target[:5000]

from sklearn.metrics import accuracy_score
print(accuracy_score(testing_rf,labels_rf))

#check for other metrics as the data is imbalanced
from sklearn import metrics
print(metrics.classification_report(testing_rf, labels_rf))
print(metrics.confusion_matrix(testing_rf,labels_rf))
print(metrics.precision_score(testing_rf,labels_rf))
print(metrics.recall_score(testing_rf,labels_rf))